In [1]:
import sys
sys.path = ['D:/projects/cleanX'] + sys.path

In [2]:
# we will need to import some libraries
import pandas as pd
import os

from cleanX import (
    dataset_processing as csvp,
    dicom_processing as dicomp,
    image_work as iwork,
)